<a href="https://colab.research.google.com/github/VinKKAP/Data-Analysis-with-LLM/blob/main/Experiment_Run1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [4]:
!git clone https://github.com/VinKKAP/Data-Analysis-with-LLM.git

Cloning into 'Data-Analysis-with-LLM'...
remote: Enumerating objects: 379, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 379 (delta 3), reused 2 (delta 2), pack-reused 375 (from 1)
Receiving objects: 100% (379/379), 57.61 MiB | 22.09 MiB/s, done.
Resolving deltas: 100% (187/187), done.
Updating files: 100% (68/68), done.


In [5]:
!pip install -r /content/Data-Analysis-with-LLM/experiment/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.0 MB/s eta 0:00:00


In [4]:
import torch
print(torch.cuda.is_available())

True


In [ ]:

'''
Correlation Prediction Script
Modified to use hardcoded parameters and handle single dataset scenarios
'''
from multiprocessing import set_start_method
try:
    set_start_method('spawn')
except RuntimeError:
    pass

import sklearn.metrics as metrics
import pandas as pd
import random as rand
import time

from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from google.colab import drive
drive.mount('/content/drive')
from google.colab import userdata
userdata.get('HF_TOKEN')
import wandb
wandb.init(project="Distilbert_run8") # Initialize wandb run


def add_type(row):
    """ Enrich column name by adding column type.

    Args:
        row: describes correlation between two columns.

    Returns:
        row with enriched column names.
    """
    row['column1'] = row['column1'] + ' ' + row['type1']
    row['column2'] = row['column2'] + ' ' + row['type2']
    return row


def def_split(data, test_ratio, seed):
    """ Split data into training and test set.

    With this approach, different column pairs from the
    same data set may appear in training and test set.

    Args:
        data: a pandas dataframe containing all data.
        test_ratio: ratio of test cases after split.
        seed: random seed for deterministic results.

    Returns:
        a tuple containing training, then test data.
    """
    print('Data sets in training and test may overlap')
    x_train, x_test, y_train, y_test = train_test_split(
      data[['column1', 'column2', 'type1', 'type2']], data['label'],
      test_size=test_ratio, random_state=seed)
    train = pd.concat([x_train, y_train], axis=1)
    test = pd.concat([x_test, y_test], axis=1)
    print(f'train shape: {train.shape}')
    print(f'test shape: {test.shape}')
    return train, test


def ds_split(data, test_ratio):
    """ Split column pairs into training and test samples.

    If only one dataset is present, this function will fall back
    to default train-test split.

    Args:
        data: a pandas dataframe containing all data.
        test_ratio: ratio of test cases after splitting.

    Returns:
        a tuple containing training, then test set.
    """
    # Check if multiple datasets are present
    unique_datasets = data['dataid'].nunique()

    if unique_datasets <= 1:
        print('Only one dataset detected. Falling back to default split.')
        # If only one dataset, use default train-test split
        return def_split(data, test_ratio, seed=42)

    print('Separating training and test sets by data')
    counts = data['dataid'].value_counts()
    nr_vals = len(counts)
    nr_test_ds = int(nr_vals * test_ratio)
    print(f'Nr. test data sets: {nr_test_ds}')

    ds_ids = counts.index.values.tolist()
    test_ds = rand.sample(ds_ids, nr_test_ds)
    print(f'TestDS: {test_ds}')

    def is_test(row):
        return row['dataid'] in test_ds

    data['istest'] = data.apply(is_test, axis=1)
    train = data[data['istest'] == False]
    test = data[data['istest'] == True]

    print(f'train.shape: {train.shape}')
    print(f'test.shape: {test.shape}')

    return train[
        ['column1', 'column2', 'type1', 'type2', 'label']], test[
            ['column1', 'column2', 'type1', 'type2', 'label']]


def baseline(col_pairs):
    """ A simple baseline predicting correlation via Jaccard similarity.

    Args:
        col_pairs: list of tuples with column names.

    Returns:
        list of predictions (1 for correlation, 0 for no correlation).
    """
    predictions = []
    for cp in col_pairs:
        c1 = cp[0]
        c2= cp[1]
        s1 = set(c1.split())
        s2 = set(c2.split())
        ns1 = len(s1)
        ns2 = len(s2)
        ni = len(set.intersection(s1, s2))
        # calculate Jaccard coefficient
        jac = ni / (ns1 + ns2 - ni)
        # predict correlation if similar
        if jac > 0.5:
            predictions.append(1)
        else:
            predictions.append(0)
    return predictions


def log_metrics(
        coeff, min_v1, max_v2, mod_type, mod_name, scenario,
        test_ratio, sub_test, test_name, lb, ub, pred_method,
        out_path, training_time):
    """ Predicts using baseline or model, writes metrics to file.

    Args:
        (multiple arguments for logging performance metrics)
    """
    sub_test.columns = [
        'text_a', 'text_b', 'type1', 'type2', 'labels', 'length', 'nrtokens']
    # print out a sample for later analysis
    print(f'Sample for test {test_name}:')
    sample = sub_test.sample(frac=0.1)
    print(sample)
    # predict correlation via baseline or model
    sub_test = sub_test[['text_a', 'text_b', 'labels']]
    samples = []
    for _, r in sub_test.iterrows():
        samples.append([r['text_a'], r['text_b']])
    s_time = time.time()
    if pred_method == 0:
        preds = baseline(samples)
    else:
        preds = model.predict(samples)[0]
    # log various performance metrics
    t_time = time.time() - s_time
    nr_samples = len(sub_test.index)
    t_per_s = float(t_time) / nr_samples
    f1 = metrics.f1_score(sub_test['labels'], preds)
    pre = metrics.precision_score(sub_test['labels'], preds)
    rec = metrics.recall_score(sub_test['labels'], preds)
    acc = metrics.accuracy_score(sub_test['labels'], preds)
    mcc = metrics.matthews_corrcoef(sub_test['labels'], preds)
    # also log to local file
    with open(out_path, 'a+') as file:
        file.write(f'{coeff},{min_v1},{max_v2},"{mod_type}",' \
                f'"{mod_name}","{scenario}",{test_ratio},' \
                f'"{test_name}",{pred_method},{lb},{ub},' \
                f'{f1},{pre},{rec},{acc},{mcc},{t_per_s},' \
                f'{training_time}\n')
    # Log metrics to W&B
    wandb.log({
        "Coefficient": coeff,
        "F1 Score": f1,
        "Precision": pre,
        "Recall": rec,
        "Accuracy": acc,
        "MCC": mcc,
        "Prediction Time per Sample": t_per_s,
        "Training Time": training_time,
        "Test Name": test_name,
        "Test Ratio": test_ratio
    })
def names_length(row):
    """ Calculate combined length of column names. """
    return len(row['text_a']) + len(row['text_b'])


def names_tokens(row):
    """ Calculates number of tokens (separated by spaces). """
    return row['text_a'].count(' ') + row['text_b'].count(' ')


def run_correlation_prediction(
    src_path,           # Path to source CSV file
    coeff='theilsu',    # Correlation coefficient
    min_v1=0.15,         # Minimal coefficient value for correlation
    max_v2=1,        # Maximal p-value for correlation
    mod_type='distilbert', # Model type
    mod_name='distilbert-base-uncased', # Specific model
    scenario='defsep',  # Splitting scenario
    test_ratio=0.2,     # Ratio of samples for testing
    use_types=1,        # Whether to use column types
    out_path='results.csv' # Output path for results
):
    # Initialize for deterministic results
    seed = 42
    rand.seed(seed)

    # Load data
    data = pd.read_csv(src_path, sep=',')
    data = data.sample(frac=1, random_state=seed)
    data.columns = [
        'dataid', 'datapath', 'nrrows', 'nrvals1', 'nrvals2',
        'type1', 'type2', 'column1', 'column2', 'method',
        'coefficient', 'pvalue', 'time']

    # Enrich column names if activated
    if use_types:
        data = data.apply(add_type, axis=1)

    # Filter data
    data = data[data['method']==coeff]
    nr_total = len(data.index)
    print(f'Nr. samples: {nr_total}')
    print('Sample from filtered data:')
    print(data.head())

    # Label data
    def coefficient_label(row):
        """ Label column pair as correlated or uncorrelated. """
        if abs(row['coefficient']) >= min_v1 and abs(row['pvalue']) <= max_v2:
            return 1
        else:
            return 0
    data['label'] = data.apply(coefficient_label, axis=1)

    # Split into test and training
    if scenario == 'defsep':
        train, test = def_split(data, test_ratio, seed)
    elif scenario == 'datasep':
        train, test = ds_split(data, test_ratio)
    else:
        raise ValueError(f'Undefined scenario: {scenario}')

    train.columns = ['text_a', 'text_b', 'type1', 'type2', 'labels']
    test.columns = ['text_a', 'text_b', 'type1', 'type2', 'labels']
    print(train.head())
    print(test.head())

    # Prepare loss scaling
    lab_counts = train['labels'].value_counts()
    nr_zeros = lab_counts.loc[0]
    nr_ones = lab_counts.loc[1]
    nr_all = float(len(train.index))
    weights = [nr_all/nr_zeros, nr_all/nr_ones]

    # Train classification model
    s_time = time.time()
    model_args = ClassificationArgs(
        num_train_epochs=5, train_batch_size=100, eval_batch_size=100,
        overwrite_output_dir=True, manual_seed=seed,
        evaluate_during_training=True, no_save=True,
        weight_decay=0.01,  # L2 regularization
        early_stopping_patience=3, # Early stopping
        )
    global model  # Make model global so it can be accessed in log_metrics
    model = ClassificationModel(
        mod_type, mod_name, weight=weights,
        use_cuda=False, args=model_args)
    model.train_model(
        train_df=train, eval_df=test, acc=metrics.accuracy_score,
        rec=metrics.recall_score, pre=metrics.precision_score,
        f1=metrics.f1_score)
    training_time = time.time() - s_time

    test['length'] = test.apply(names_length, axis=1)
    test['nrtokens'] = test.apply(names_tokens, axis=1)

    # Initialize result file
    with open(out_path, 'w') as file:
        file.write(
            'coefficient,min_v1,max_v2,mod_type,mod_name,scenario,test_ratio,'
            'test_name,pred_method,lb,ub,f1,precision,recall,accuracy,mcc,'
            'prediction_time,training_time\n')

    # Use simple baseline and model for prediction
    for m in [0, 1]:
        # Use entire test set
        test_name = f'{m}-final'
        log_metrics(
            coeff, min_v1, max_v2, mod_type, mod_name, scenario,
            test_ratio, test, test_name, 0, 'inf', m, out_path, training_time)

        # Test for data types
        for type1 in ['object', 'float64', 'int64', 'bool']:
            for type2 in ['object', 'float64', 'int64', 'bool']:
                sub_test = test.query(f'type1=="{type1}" and type2=="{type2}"')
                if sub_test.shape[0]:
                    test_name = f'Types{type1}-{type2}'
                    log_metrics(
                        coeff, min_v1, max_v2, mod_type, mod_name, scenario,
                        test_ratio, sub_test, test_name, -1, -1, m,
                        out_path, training_time)

        # Test for different subsets
        for q in [(0, 0.25), (0.25, 0.5), (0.5, 1)]:
            qlb = q[0]
            qub = q[1]
            # Column name length
            lb = test['length'].quantile(qlb)
            ub = test['length'].quantile(qub)
            sub_test = test[(test['length'] >= lb) & (test['length'] <= ub)]
            test_name = f'L{m}-{qlb}-{qub}'
            log_metrics(
                coeff, min_v1, max_v2, mod_type, mod_name, scenario,
                test_ratio, sub_test, test_name, lb, ub, m, out_path, training_time)

            # Number of tokens in column names
            lb = test['nrtokens'].quantile(qlb)
            ub = test['nrtokens'].quantile(qub)
            sub_test = test[(test['nrtokens'] >= lb) & (test['nrtokens'] <= ub)]
            test_name = f'N{m}-{qlb}-{qub}'
            log_metrics(
                coeff, min_v1, max_v2, mod_type, mod_name, scenario,
                test_ratio, sub_test, test_name, lb, ub, m, out_path, training_time)


# Example usage
if __name__ == '__main__':
    # Modify these parameters as needed
    run_correlation_prediction(
        src_path='/content/drive/My Drive/Colab Notebooks/Liter/correlations/corresult5.csv',
        coeff='theilsu',
        min_v1=0.15,
        max_v2=1,
        mod_type='distilbert',
        mod_name='distilbert-base-uncased',
        scenario='defsep',
        test_ratio=0.2,
        use_types=1,
        out_path='/content/drive/My Drive/Colab Notebooks/Liter/correlations/results/results8.csv'
    )

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Nr. samples: 2556
Sample from filtered data:
      dataid                 datapath  nrrows  nrvals1  nrvals2   type1  \
1988       2  ../data/output_file.csv      63        6       21  object   
1216       2  ../data/output_file.csv      63        2        2  object   
1385       2  ../data/output_file.csv      63       38       22  object   
1334       2  ../data/output_file.csv      63       21        2  object   
175        2  ../data/output_file.csv      63        4        4  object   

       type2                              column1  \
1988  object      detailed_class_of_worker object   
1216  object                   school_type object   
1385   int64  detailed_second_degree_field object   
1334  object           second_degree_field object   
175   object                marital_status object   

                             column2   method  coefficient    pvalue      time  
1988               birthplace object  theilsu     0.544940  0.257974  0.000240  
1216  detailed_grade_at

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/21 [00:00<?, ?it/s]